## import package

In [ ]:
import scipy.io
import numpy as np
import pandas as pd


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from mord import LogisticIT, LogisticAT

from pyentrp import entropy as ent

In [ ]:
#visualization
def confusion_heat_map(model, X, y):
    y_fit = model.predict(X)
    import seaborn as sns
    from matplotlib import pyplot as plt
    from sklearn.metrics import confusion_matrix
    mat = confusion_matrix(y, y_fit, normalize = 'true')
    sns.heatmap(mat, square=True, annot=True, fmt='.2%', cbar='Blues',
                xticklabels=np.unique(y),
                yticklabels=np.unique(y))

    plt.ylabel('true label')
    plt.xlabel('predicted label');
    plt.show()

    

## load train and test data

In [ ]:
df_5 = pd.read_csv('D:/BCI_Classification/bci_fft_5freq_data_train.csv')

X = df_5[['20','21','22','23','40','41','42','43']]
y = df_5['y']
day = df_5['day']

In [ ]:
np.unique(y)

In [ ]:
np.unique(day)


## logistic regression

In [ ]:
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify = day)


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X_train, y_train)

print(clf.score(X_test, y_test))
print('coefficient: '+str(clf.coef_))

confusion_heat_map(clf, X, y)

## svm, grid search

In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify = day)


#classification
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

#pca = PCA(n_components=2, whiten=True)
svc = SVC(kernel='rbf')
model = make_pipeline(svc)

from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid,cv = 5)

%time grid.fit(X_train, y_train)
print(grid.best_params_)


#scoring
model = grid.best_estimator_
y_fit = model.predict(X_test)


confusion_heat_map(model, X, y)

In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify = day)


#scoring: svm best:{'svc__C': 10, 'svc__gamma': 0.005}
#pca = PCA(n_components=2, whiten=True)
svc = SVC(kernel='rbf', C = 10, gamma = 0.005)
model = make_pipeline(pca, svc)

model.fit(X_train, y_train)
y_fit = model.predict(X_test)
print(model.score(X_test, y_test))

#visualization
confusion_heat_map(model, X, y)

## random forest

In [ ]:
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

from sklearn.tree import DecisionTreeClassifier
#pca = PCA(n_components=2, whiten=True)
tree = DecisionTreeClassifier()
model = make_pipeline(tree)

model.fit(X_train, y_train)
print('test score:{}'.format(model.score(X_test, y_test)))

confusion_heat_map(model, X, y)

## grid search decision tree

In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

pca = PCA(n_components=2, whiten=True)
tree = DecisionTreeClassifier()
model = make_pipeline(pca, tree)

parameter_grid = {'decisiontreeclassifier__criterion': ['gini', 'entropy'],
                  'decisiontreeclassifier__splitter': ['best', 'random'],
                  'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5],
                  'decisiontreeclassifier__max_features': [1, 2]}

cross_validation = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(model, param_grid=parameter_grid, cv=cross_validation)

grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

grid_search = grid_search.best_estimator_
print('score for test set: {}'.format(grid_search.score(X_test, y_test)))

cv_scores = cross_val_score(grid_search, X, y, cv=20)
sns.distplot(cv_scores)
plt.title('Average score: {}'.format(np.mean(cv_scores)))
plt.show()

## random tree classifier

In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
## random forest
from sklearn.ensemble import RandomForestClassifier


pca = PCA(n_components=2, whiten=True)
tree = RandomForestClassifier(n_estimators=100)
model = make_pipeline(pca, tree)

parameter_grid = {'randomforestclassifier__criterion': ['gini', 'entropy'],
                  'randomforestclassifier__max_depth': [1, 2, 3, 4, 5],
                  'randomforestclassifier__max_features': [1, 2]}

cross_validation = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(model, param_grid=parameter_grid, cv=cross_validation)

%time grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

grid_search = grid_search.best_estimator_
print('score for test set: {}'.format(grid_search.score(X_test, y_test)))


In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


## random forest
from sklearn.ensemble import RandomForestClassifier

pca = PCA(n_components=2, whiten=True)
tree = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=5, max_features=1)
model = make_pipeline(pca, tree)

model.fit(X_train, y_train)
print('score for test set: {}'.format(model.score(X_test, y_test)))

confusion_heat_map(grid_search, X_train, y_train)

## random forest without pca

In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import StratifiedKFold
## random forest random searchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
criterion = ['gini','entorpy']
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, 
                               verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
%time rf_random.fit(X_train, y_train)
  

print('Best score: {}'.format(rf_random.best_score_))
print('Best parameters: {}'.format(rf_random.best_params_))

rf_random = rf_random.best_estimator_
print('score for test set: {}'.format(rf_random.score(X_test, y_test)))


In [ ]:
rf_random.get_params

In [ ]:
## Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


## random forest
from sklearn.ensemble import RandomForestClassifier
# 3 levels
# tree = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
#                        max_depth=40, max_features='auto', max_leaf_nodes=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=400,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)

# 5 levels
tree = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

tree.fit(X_train, y_train)
print('score for test set: {}'.format(tree.score(X_test, y_test)))

confusion_heat_map(tree, X_test, y_test)

In [ ]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, random_state=4)


confusion_heat_map(tree, X_test, y_test)

## cross day validation (0% calibration)

In [ ]:
for i in range(1,5):#select day
    chose_day = i

    X_train = X[day != chose_day]
    X_test = X[day ==  chose_day]

    y_train = y[day!=chose_day]
    y_test = y[day == chose_day]

#random forest
    tree.fit(X_train, y_train)
    
    print('day:{}'.format(i))
    print('score for test set: {}'.format(tree.score(X_test, y_test)))
    confusion_heat_map(tree, X_test, y_test)

## cross day validation(20% of calibration)

In [ ]:
for i in range(1,6):#select day
    chose_day = i

    X_train = X[day != chose_day]
    X_test = X[day ==  chose_day]

    y_train = y[day!=chose_day]
    y_test = y[day == chose_day]

    #include part of the test
    X_pre_train, X_test, y_pre_train, y_test = train_test_split(X_test, y_test, test_size=0.9, random_state=4)

    X_train = pd.concat([X_train, X_pre_train], axis = 0)
    y_train = np.concatenate([y_train, y_pre_train])


    ## random forest
    tree.fit(X_pre_train, y_pre_train)
    
    print('day:{}'.format(i))
    print('score for test set: {}'.format(tree.score(X_test, y_test)))
    confusion_heat_map(tree, X_test, y_test)

### cross day validation (some levels)

In [ ]:
for i in range(1,6):#select day
    chose_day = i

    X_train = X[day != chose_day]
    X_test = X[day == chose_day]

    y_train = y[day!=chose_day]
    y_test = y[day == chose_day]

    #include part of the test
    X_pre_train = X_test[(y_test==2) | (y_test==4)| (y_test==5)| (y_test==6)]
    y_pre_train = y_test[(y_test==2) | (y_test==4)| (y_test==5)| (y_test==6)]
        
    X_test = X_test[(y_test!=2) & (y_test!=4) & (y_test!=5) & (y_test!=6)]    
    y_test = y_test[(y_test!=2) & (y_test!=4) & (y_test!=5) & (y_test!=6)]  
    
    X_train = pd.concat([X_train, X_pre_train], axis = 0)
    y_train = np.concatenate([y_train, y_pre_train])


    ## random forest
    tree.fit(X_train, y_train)
    
    print('day:{}'.format(i))
    print('score for test set: {}'.format(tree.score(X_test, y_test)))
    confusion_heat_map(tree, X_test, y_test)

## simple neural network

In [ ]:
# from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
# from tensorflow.keras.constraints import max_norm
# import os
# import glob
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.constraints import max_norm

# Function to create model, required for KerasClassifier
def create_model():
    model = models.Sequential() 
    model.add(layers.Flatten(input_shape=(1, 4)))
    
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.2))

    model.add(layers.Dense(256,activation='relu'))
    model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(256,activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(5, activation='softmax'))

    #model compilation
    #Compile and train the model
    model.compile(optimizer =keras.optimizers.Adam(lr = 0.01),
                      loss = 'sparse_categorical_crossentropy',
                      metrics=['acc'], verbose = 1)
    
    return model



In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(neuron1=64, neuron2 = 256, neuron3 = 256, learn_rate=0.01, optimizer='adam'):
    model = models.Sequential() 
    model.add(layers.Flatten(input_shape=(1, 4)))
    
    model.add(layers.Dense(neuron1,activation='relu'))
    model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.2))

    model.add(layers.Dense(neuron2,activation='relu'))
    model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(neuron3,activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(5, activation='softmax'))

    #model compilation
    #Compile and train the model
    model.compile(optimizer = optimizer,
                      loss = 'sparse_categorical_crossentropy',
                      metrics=['acc'], verbose = 1)
    
    return model



# create model
model = KerasClassifier(build_fn=create_model,epochs = 200, verbose=0)
# define the grid search parameters
batch_size = [30, 60, 100]
optimizer = ['SGD', 'RMSprop', 'adam']
learn_rate = [0.001, 0.01, 0.1]
neuron1 = [64,128,256]
neuron2 = [64,128,256]
neuron3 = [64,128,256]



param_grid = dict(batch_size=batch_size,learn_rate=learn_rate, optimizer=optimizer,
                 neuron1 = neuron1, neuron2 = neuron2, neuron3 = neuron3)

# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, 
#                                verbose=2, random_state=42, n_jobs = -1)

grid = RandomizedSearchCV(estimator=model, param_grid=param_grid, cv=5, n_iter = 100, random_state = 42, n_jobs = -1)

X_train = np.expand_dims(X,axis=1)
y_train = y.values - 2


grid_result = grid.fit(X_train, y_train)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=2, stratify = y)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.2, random_state=2,stratify = y_train )


X_train = np.expand_dims(X_train,axis=1)
X_test = np.expand_dims(X_test,axis=1)
X_val = np.expand_dims(X_val,axis=1)

y_train = y_train.values - 2
y_test = y_test.values - 2
y_val = y_val.values - 2 

# checkpoint
filepath="D://BCI_Classification//model_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
model = create_model()
history = model.fit(X_train, y_train,epochs=200, validation_data=(X_val, y_val), callbacks=callbacks_list)
    
#evaluation
model.load_weights(filepath)
test_loss,test_acc = model.evaluate(X_test,y_test)

print('Model evaluation:{}'.format(test_acc))



In [ ]:
from sklearn.model_selection import KFold

day_value = day.values

for train_day, predict_day in  KFold(5).split([0,1,2,3,4]):
    
    day_predict_idx = np.unique(day_value)[predict_day]
    print(day_predict_idx)
    train_list = (day_value != day_predict_idx)
    
    #training set
    X_train = X.iloc[train_list==True , :]
    y_train = y[train_list==True]
       
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=2, stratify = y)
    
    #test set
    X_test= X.iloc[train_list == False, :]
    y_test= y[train_list==False]

    X_train = np.expand_dims(X_train,axis=1)
    X_test = np.expand_dims(X_test,axis=1)
    X_val = np.expand_dims(X_val,axis=1)

    y_train = y_train.values - 2
    y_test = y_test.values - 2
    y_val = y_val.values - 2 

    # checkpoint
    filepath="D://BCI_Classification//day_"+str(day_predict_idx[0])+"_model_weights.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    #Compile and train the model
    model = create_model()
    history = model.fit(X_train, y_train,epochs=100, validation_data=(X_val, y_val), callbacks=callbacks_list)

    #evaluation
    model.load_weights(filepath)
    test_loss,test_acc = model.evaluate(X_test,y_test)

    print('Model evaluation:{}'.format(test_acc))



In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label ='val_loss' )
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


In [ ]:
for i in range(1,6):
    filepath="D://BCI_Classification//day_"+str(i)+"_model_weights.hdf5"
    #evaluation
    model.load_weights(filepath)
    
    X_train = X[day!=i]
    y_train = y[day!=i]
    X_train = np.expand_dims(X_train,axis=1)
    y_train = y_train.values - 2
    
    

    X_test = X[day==i]
    y_test = y[day==i]
    X_test = np.expand_dims(X_test,axis=1)
    y_test = y_test.values - 2
    
    
    train_loss,train_acc = model.evaluate(X_train,y_train, verbose = 0)
    test_loss,test_acc = model.evaluate(X_test,y_test, verbose = 0)
    print('Predict day:{}'.format(i))
    print('training set:{}'.format(train_acc))
    print('test set:{}'.format(test_acc))
    print()